<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [62]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')
df_codigos.head()

,codigo_iso,pais
0,AFG,Afghanistán
1,AGO,Angola
2,ALB,Albania
3,AND,Andorra
4,ARE,Emiratos Árabes Unidos




### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [63]:
# FIXME
#a
df_anio_1=pd.read_csv(archivos_anio[0])
df_anio_2=pd.read_csv(archivos_anio[1])
df_anio=pd.concat([df_anio_1,df_anio_2], ignore_index=True)
df_anio.columns = df_anio.columns.str.lower()
#b
dupe = df_codigos[df_codigos.duplicated(subset=['codigo_iso'], keep=False)]
print("Se encontró el siguiente código ISO asociado a dos países:")
print(dupe)
bad_index = df_codigos[df_codigos['pais'] == 'malo'].index
df_codigos=df_codigos.drop(bad_index)
print(df_codigos.tail()) #efectivamente lo elimina
#c
df = pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner')
#la forma inner funciona a modo de "intersección" entre "codigo_iso" por lo que se asegura lo pedido por como funciona la intersección.
df.head()

Se encontró el siguiente código ISO asociado a dos países:
    codigo_iso      pais
179        ZWE  Zimbabue
180        ZWE      malo
    codigo_iso       pais
175        WSM      Samoa
176        YEM      Yemen
177        ZAF  Sudáfrica
178        ZMB     Zambia
179        ZWE   Zimbabue


,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [61]:
# FIXME
#Estructura df
print("El data frame tiene la siguiente cantidad de filas y columnas respectivamente: ",df.shape[0],df.shape[1])
print("El nombre de las columnas son: ",df.columns.tolist())
print("El tipo de datos de las columnas aparecen en la siguiente tabla: ")
df.info()
print("Como se vió en la tabla anterior. Los datos de codigo_iso y pais son object, es decir str, lo cual está bien. \nLuego tanto indice como ranking son flotantes, ya que esta columna debe representar numeros tiene sentido, sin embargo \nsegún el diccionario de variables ranking debería ser un entero como lo es anio, el cual como representa años y es un numero entero tiene bastante sentido. \nEn resumen no parece haber algún tipo de dato inesperado.")
#Resumen estadistico df
print(df.describe())
print("Podemos notar que los valores maximos tanto de indice como de ranking son absurdamente altos \n(lo cual muy probablemente se trate de algun error pues no tiene sentido un ranking de mas de 100 mil o un indice de mas de 60 mil)")
print("El indice minimo es:", df['indice'].min())
print("El indice maximo es:", df['indice'].max())
print("El promedio de los indices es:", df['indice'].mean())
max_index=df.loc[df['indice'].idxmax()]
min_index=df.loc[df['indice'].idxmin()]
print("País con el máximo índice es: ", max_index[['pais','anio','indice']])
print("País con el mínimo índice es: ", min_index[['pais','anio','indice']])
max_rank=df.loc[df['ranking'].idxmax()]
min_rank=df.loc[df['ranking'].idxmin()]
print("País con el máximo ranking es: ", max_rank[['pais','anio','ranking']]) #Qué está pasando en kosovo como para que una distopía tipo 1984 probablemente tenga mas libertad de prensa
print("País con el mínimo ranking es: ", min_rank[['pais','anio','ranking']]) #Fuerza kosovo
#Datos faltantes df
print("Hay \n", df.isnull().sum(), "\ncantidad de valores nulos por columna.")
print("La cantidad de filas con al menos un valor faltante es: ", df.isnull().any(axis=1).sum())
print("La proporción de filas que tienen al menos un valor faltante es: ", df.isnull().any(axis=1).sum()/len(df), "es decir un 12.9% aproximadamente de las filas tiene al menos un valor nulo")
porcentaje_col = (df.isnull().sum() / len(df)) * 100
col_over_30 = porcentaje_col[porcentaje_col > 30]
if not col_over_30.empty:
    print("Las siguientes columnas tienen más del 30% de datos faltantes:")
    print(col_over_30.round(2).astype(str) + '%')
else:
    print("Ninguna columna supera el 30% de datos faltantes.")
#Unicidad y duplicados de df
print("Hay", df['pais'].nunique(), "países distintos")
print("Hay", df['anio'].nunique(), "años distintos")
if df.duplicated().sum() > 0:
    print("Sí, se encontraron",df.duplicated().sum() ,"filas duplicadas.")
else:
    print("No existen filas completamente duplicadas en el DataFrame.")
conteo = df.groupby('codigo_iso')['pais'].nunique() > 1
iso_repetido = conteo[conteo].index.tolist()
df_inconsistente = df[df['codigo_iso'].isin(iso_repetido)]
print(df_inconsistente[['codigo_iso', 'pais']].drop_duplicates()) #No parece haber codigos duplicados, pues se obtiene una lista vacía

El data frame tiene la siguiente cantidad de filas y columnas respectivamente:  3060 5
El nombre de las columnas son:  ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']
El tipo de datos de las columnas aparecen en la siguiente tabla: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  3060 non-null   object 
 1   anio        3060 non-null   int64  
 2   indice      2664 non-null   float64
 3   ranking     2837 non-null   float64
 4   pais        3060 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 119.7+ KB
Como se vió en la tabla anterior. Los datos de codigo_iso y pais son object, es decir str, lo cual está bien. 
Luego tanto indice como ranking son flotantes, ya que esta columna debe representar numeros tiene sentido, sin embargo 
según el diccionario de variables ranking debería ser un entero como lo es 




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [69]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN'] #USA en latam?

df_america =  df[df['codigo_iso'].isin(america)]
anios_unicos = df_america['anio'].unique().tolist()
for i in anios_unicos:
  df_anio=df_america[df_america['anio'==i]]
  if df_anio.empty:
      print(f"Año: {i} - No hay datos disponibles para este año.")
      continue
  max_index=df_anio.loc[df_anio['indice'].idxmax()]
  min_index=df_anio.loc[df_anio['indice'].idxmin()]
  print("El país con la mayor libertad de prensa del año",i,"es",min_index['pais'],"con un indice de",min_index['indice'])

KeyError: False

### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [ ]:
# FIX ME